# What is about ? 

Here we show various visualizations for single-cell data file obtained from sciPlex2 with zero dose.

We load data matrix in sparse format and add informations saved in other files.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import time
t0_start = time.time()

import matplotlib.pyplot as plt
import seaborn as sns

# Load expression matrix in sparse format


3500 cells x 58347 genes 

In [ ]:
import scipy.sparse
fn = '/kaggle/input/scrnaseq-exposed-to-multiple-compounds/sciPlex2_zero_dose_sparse.npz'
sparse_matrix = scipy.sparse.load_npz(fn)
type(sparse_matrix), sparse_matrix.shape

In [ ]:
print(sparse_matrix[:10,:8].toarray())

# Load genes names 

58347 genes

In [ ]:
fn = '/kaggle/input/scrnaseq-exposed-to-multiple-compounds/genes_sciPlex2.csv'
genes = pd.read_csv(fn, index_col = 0)
genes

# Start processing

# Log and normalization 

In [ ]:
m = sparse_matrix.copy()
if 1:
    m2 = m.copy()
    m2.data = np.log(1+m2.data)
    m2 = m2.multiply( 1/m2.sum(axis=1))
    m2 = m2.tocsr()
    str_preproc = 'LogThenNorm'
else:
    m2 = m.multiply( 1/m.sum(axis=1))
    m2 = m2.tocsr()
    m2.data = np.log(1+m2.data)
    str_preproc = 'NormThenLog'

print(str_preproc)#  
print(m2[:4,:4].toarray())
print(m2.sum(axis = 1)[:4])

# Filter genes by variance  - take top 10 000 


in our case filtering by variance is more or less the same as by expression sum

there is NO built-in variance calculation for sparse matrices in python, so we calculate by hands 

## Calculate variance

In [ ]:
# Show that expression and variance are both growing 

t = m2.copy()
t.data **= 2#.toarray()
#print( t.toarray() )
X_var = np.array(t.mean(axis = 0 )).ravel() - np.array(m2.mean(axis = 0) ).ravel()**2  

genes['Variance'] = X_var
#plt.plot( np.sort( (  genes['Variance'].values) ))

genes['Expression Sum'] = np.asarray(m2.sum(axis = 0)).ravel()

g2 = genes.sort_values('Expression Sum')
g2 = genes.sort_values('Variance')

plt.plot(np.log10(1+ g2['Expression Sum'].values) )
# plt.plot(g2['Variance'].values)
plt.title('Expression sum')
plt.show()
plt.plot(np.log10(1+ g2['Variance'].values) )
plt.title('Variance')
plt.show()

## prepare 10000 indexes

In [ ]:
t = m2.copy()
t.data **= 2#.toarray()
#print( t.toarray() )
X_var = np.array(t.mean(axis = 0 )).ravel() - np.array(m2.mean(axis = 0) ).ravel()**2  
IX = np.argsort(X_var)
IX = IX[-10000:]


## do filtering 

In [ ]:
m = m[:,IX]
m2 = m2[:,IX]
genes = genes.iloc[IX,:]

genes.shape, m2.shape, m.shape

In [ ]:
genes.shape, m2.shape, m.shape

# Visualization - UMAP ( preceded by svd to 50 dimensions) 


Also droping out 10 most expressed components

In [ ]:
if 1:
    import seaborn  as sns
    from sklearn.decomposition import TruncatedSVD
    import time
    import umap

    reducer = umap.UMAP(random_state=42)
    str_reducer = 'UMAP'
    #reducer = ncvis.NCVis()
    #reducer =  trimap.TRIMAP()
    svd = TruncatedSVD( n_components=50, n_iter=7, random_state=42)


    v = np.asarray(m2.sum(axis=0)).ravel()
    IX = np.argsort(v )

    t00 = time.time()
    for count_made_zeros in [10]: # range(10,100,10):

        m3 = m2.copy()#  != 0 # 
        if count_made_zeros != 0:
            #m3[:,IX[-count_made_zeros:]] = 0
            m3 = m3[:,IX[:-count_made_zeros]] # droping columns - same as make them zeros , but does not create memory alloc prob


        t0 = time.time()
        r = svd.fit_transform(m3)
        print(time.time() - t0, 'seconds passed on svd')
        t0 = time.time()
        r = reducer.fit_transform(r)
        print(time.time() - t0, 'seconds passed on ', str_reducer)

        fig = plt.figure(figsize = (20,12))
        c = 0; 

        #c+=1; fig.add_subplot(1, 2 , c)         
        #color_by = df['drug'].copy()
        sns.scatterplot(x=r[:,0],y=r[:,1] ) # ,   hue= color_by, alpha = 0.7, marker = '.')
        plt.title(str_reducer + '  count_made_zeros='  +str( count_made_zeros ) + ' Non binarized' )


        print(time.time() - t0, 'seconds passed')

    print('Finished.', time.time() - t00, 'seconds passed total ')    

# Visualuzations by many methods 

In [ ]:
!pip install ncvis
import ncvis
vis = ncvis.NCVis()

!pip install trimap 
import trimap
trimap1 = trimap.TRIMAP()

In [ ]:
from sklearn.decomposition import TruncatedSVD
import time

t00 = time.time()

if 1: # drop top expressed genes
    count_drop_top_expressed_genes = 10 # ]: # range(10,100,10):
    v = np.asarray(m2.sum(axis=0)).ravel()
    IX = np.argsort(v )
    m3 = m2.copy()#  != 0 # 
    if count_made_zeros != 0:
        m3 = m3[:,IX[:-count_drop_top_expressed_genes]] # droping columns - same as make them zeros , but does not create memory alloc prob

t0 = time.time()
svd = TruncatedSVD( n_components=50, n_iter=7, random_state=42)
X = svd.fit_transform(m3)
print(time.time() - t0, 'seconds passed on svd')
t0 = time.time()


from sklearn import manifold
from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import NMF
from sklearn.decomposition import FastICA
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.random_projection import SparseRandomProjection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD


from collections import OrderedDict
from functools import partial
from matplotlib.ticker import NullFormatter



n_neighbors = 10
n_components = 2
# Set-up manifold methods
LLE = partial(manifold.LocallyLinearEmbedding,
              n_neighbors, n_components, eigen_solver='auto')

methods = OrderedDict()
methods['PCA'] = PCA()
methods['umap'] = umap.UMAP(n_components = n_components)
methods['t-SNE'] = manifold.TSNE(n_components=n_components, init='pca', random_state=0)
methods['NCIvis'] = ncvis.NCVis()
methods['trimap'] = trimap.TRIMAP()
methods['ICA'] = FastICA(n_components=n_components,         random_state=0)
methods['FA'] = FactorAnalysis(n_components=n_components, random_state=0)
methods['LLE'] = LLE(method='standard')
methods['Modified LLE'] = LLE(method='modified')
methods['Isomap'] = manifold.Isomap(n_neighbors, n_components)
methods['MDS'] = manifold.MDS(n_components, max_iter=100, n_init=1)
methods['SE'] = manifold.SpectralEmbedding(n_components=n_components,
                                           n_neighbors=n_neighbors)
methods['NMF'] = NMF(n_components=n_components,  init='random', random_state=0) 
methods['RandProj'] = SparseRandomProjection(n_components=n_components, random_state=42)

rand_trees_embed = make_pipeline(RandomTreesEmbedding(n_estimators=200, random_state=0, max_depth=5), TruncatedSVD(n_components=n_components) )
methods['RandTrees'] = rand_trees_embed
methods['LatDirAll'] = LatentDirichletAllocation(n_components=n_components,  random_state=0)
methods['LTSA'] = LLE(method='ltsa') 
methods['Hessian LLE'] = LLE(method='hessian') 

list_fast_methods = ['PCA','umap','FA', 'ICA','NMF','RandProj','RandTrees']
list_slow_methods = ['t-SNE','LLE','Modified LLE','Isomap','MDS','SE','LatDirAll','LTSA','Hessian LLE']

# transformer = NeighborhoodComponentsAnalysis(init='random',  n_components=2, random_state=0) # Cannot be applied since supervised - requires y 
# methods['LinDisA'] = LinearDiscriminantAnalysis(n_components=n_components)# Cannot be applied since supervised - requires y 


# Create figure
fig = plt.figure(figsize=(25, 16))

# Plot results
c = 0
for i, (label, method) in enumerate(methods.items()):
    #if label in ['umap']: continue
    if 0:
        if label not in  list_fast_methods :
            continue
        
    t0 = time.time()
    try:
        r = method.fit_transform(X.copy())
    except:
        print('Got Exception', label )
        continue 
    t1 = time.time()
    print("%s: %.2g sec" % (label, t1 - t0))
    
    if 1:
        c+=1
        fig.add_subplot(2, 3 , c) 
        sns.scatterplot(x=r[:,0],y=r[:,1] )# , hue= y)#  cell_lines_tissues_top_truncated, style =cell_lines_tissues_top_truncated , s = 250 )# df['drug'] ,  style=df['dose'])
        plt.title(label )
        plt.legend('')
        if c == 6:
            plt.show()
            fig = plt.figure(figsize=(25, 16))
            c = 0

    else:
        plt.figure(figsize = (20,12))
        sns.scatterplot(x=r[:,0],y=r[:,1] , hue= y)# cell_lines_tissues_top_truncated, style =cell_lines_tissues_top_truncated , s = 250 )# df['drug'] ,  style=df['dose'])
        plt.title(label)# 
        plt.show()

# Visualizations on subset of cell cycle  genes 

## Load list of cell cycle genes

In [ ]:
url = 'https://raw.githubusercontent.com/chervov/genes/main/cell_cycle_genes_reactome_extended.txt'
df_list_genes_cc = pd.read_csv(url, header = None)
df_list_genes_cc

In [ ]:
genes.head(1)

## Create List of positions of cell cycle genes

In [ ]:
mask = genes['1'].isin( list( df_list_genes_cc[0].values) )
IX_cc = np.where(mask > 0)[0]

IX_cc, 
len(IX_cc)

In [ ]:
m2.shape

In [ ]:
from sklearn.decomposition import TruncatedSVD
import time

t00 = time.time()

m3 = m2.copy()#  != 0 # 
m3 = m3[:,IX_cc]

if 0: # drop top expressed genes
    count_drop_top_expressed_genes = 10 # ]: # range(10,100,10):
    v = np.asarray(m2.sum(axis=0)).ravel()
    IX = np.argsort(v )
    if count_made_zeros != 0:
        m3 = m3[:,IX[:-count_drop_top_expressed_genes]] # droping columns - same as make them zeros , but does not create memory alloc prob

t0 = time.time()
svd = TruncatedSVD( n_components=50, n_iter=7, random_state=42)
X = svd.fit_transform(m3)
print(time.time() - t0, 'seconds passed on svd')
t0 = time.time()


from sklearn import manifold
from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import NMF
from sklearn.decomposition import FastICA
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.random_projection import SparseRandomProjection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD


from collections import OrderedDict
from functools import partial
from matplotlib.ticker import NullFormatter



n_neighbors = 10
n_components = 2
# Set-up manifold methods
LLE = partial(manifold.LocallyLinearEmbedding,
              n_neighbors, n_components, eigen_solver='auto')

methods = OrderedDict()
methods['PCA'] = PCA()
methods['umap'] = umap.UMAP(n_components = n_components)
methods['t-SNE'] = manifold.TSNE(n_components=n_components, init='pca', random_state=0)
methods['NCIvis'] = ncvis.NCVis()
methods['trimap'] = trimap.TRIMAP()
methods['ICA'] = FastICA(n_components=n_components,         random_state=0)
methods['FA'] = FactorAnalysis(n_components=n_components, random_state=0)
methods['LLE'] = LLE(method='standard')
methods['Modified LLE'] = LLE(method='modified')
methods['Isomap'] = manifold.Isomap(n_neighbors, n_components)
methods['MDS'] = manifold.MDS(n_components, max_iter=100, n_init=1)
methods['SE'] = manifold.SpectralEmbedding(n_components=n_components,
                                           n_neighbors=n_neighbors)
methods['NMF'] = NMF(n_components=n_components,  init='random', random_state=0) 
methods['RandProj'] = SparseRandomProjection(n_components=n_components, random_state=42)

rand_trees_embed = make_pipeline(RandomTreesEmbedding(n_estimators=200, random_state=0, max_depth=5), TruncatedSVD(n_components=n_components) )
methods['RandTrees'] = rand_trees_embed
methods['LatDirAll'] = LatentDirichletAllocation(n_components=n_components,  random_state=0)
methods['LTSA'] = LLE(method='ltsa') 
methods['Hessian LLE'] = LLE(method='hessian') 

list_fast_methods = ['PCA','umap','FA', 'ICA','NMF','RandProj','RandTrees']
list_slow_methods = ['t-SNE','LLE','Modified LLE','Isomap','MDS','SE','LatDirAll','LTSA','Hessian LLE']

# transformer = NeighborhoodComponentsAnalysis(init='random',  n_components=2, random_state=0) # Cannot be applied since supervised - requires y 
# methods['LinDisA'] = LinearDiscriminantAnalysis(n_components=n_components)# Cannot be applied since supervised - requires y 


# Create figure
fig = plt.figure(figsize=(25, 16))

# Plot results
c = 0
for i, (label, method) in enumerate(methods.items()):
    #if label in ['umap']: continue
    if 0:
        if label not in  list_fast_methods :
            continue
        
    t0 = time.time()
    try:
        r = method.fit_transform(X.copy())
    except:
        print('Got Exception', label )
        continue 
    t1 = time.time()
    print("%s: %.2g sec" % (label, t1 - t0))
    
    if 1:
        c+=1
        fig.add_subplot(2, 3 , c) 
        sns.scatterplot(x=r[:,0],y=r[:,1] )# , hue= y)#  cell_lines_tissues_top_truncated, style =cell_lines_tissues_top_truncated , s = 250 )# df['drug'] ,  style=df['dose'])
        plt.title(label )
        plt.legend('')
        if c == 6:
            plt.show()
            fig = plt.figure(figsize=(25, 16))
            c = 0

    else:
        plt.figure(figsize = (20,12))
        sns.scatterplot(x=r[:,0],y=r[:,1] , hue= y)# cell_lines_tissues_top_truncated, style =cell_lines_tissues_top_truncated , s = 250 )# df['drug'] ,  style=df['dose'])
        plt.title(label)# 
        plt.show()

In [ ]:
print(genes['1'].nunique())
print(genes['1'].value_counts().head(1) )

# Avoid duplicate gene names

among all 10 000 only MALAT1 repeated twice - with different ensemble ID , so we correct it by adding part of Ensemble ID to second gene symbol

In [ ]:
# Technical renaming to fit previous notebooks style 
genes.columns = [ 0, 1, genes.columns[2], genes.columns[3] ]
genes.columns

In [ ]:
IX = np.where( genes[1] =='MALAT1' ) [0] 
print( genes.iloc[IX,:] )

IX = np.where( genes[0] =='ENSG00000278217.1' )[0][0]
genes[1].iat[IX] = 'MALAT1_AP000769' # This gene in somehow related (part of?) to MALAT1 -  
# see e.g. http://www.ensembl.org/Homo_sapiens/Gene/Summary?db=core;g=ENSG00000278217;r=11:65502914-65503008;t=ENST00000618249
# But to avoid name duplicates we rename it differently from existing MALAT1 - ENSG00000251562.7

In [ ]:
# Check deduplicating worked okay 
print(genes[1].nunique())
print(genes[1].value_counts().head(1) )

# Markers of cell cycle with SCANPY 

In [ ]:
!pip install scanpy

In [ ]:
import anndata as ad
import scanpy as sc

In [ ]:
obs = pd.DataFrame( range(m2.shape[0] ) )

In [ ]:
print( genes.shape,m2.shape,m.shape)# , df.shape )
#df.head(1)
# obs = df.copy() # 
obs = pd.DataFrame( range(m2.shape[0] ) )

display(genes.head(1))
var = genes.reset_index().set_index(1) # Create "var" dataframe for anndat
var.columns = ['Old Index','Ensemble Id', 'Variance', 'Expression Sum']
var.index.name = 'Gene'
display(var.head(2))
adata = ad.AnnData( m2.toarray(), obs=obs, var=var ) # memory consumed less than 1G
if 0: # Examples 
    adata.obs['cell_groups'] = anno['cell_groups']  # categorical annotation of type pandas.Categorical
    adata.obs['time'] = anno['time']                # numerical annotation of type float
    # alternatively, you could also set the whole dataframe
    # adata.obs = anno

adata 

## Prepare list of phase marker genes 

See detailes here: https://www.kaggle.com/alexandervc/cell-cycle-markers-compare 

Based on Tirosh 2016 and Scanpy cell cycle tutorial 


In [ ]:
s_genes = ['MCM5', 'PCNA', 'TYMS', 'FEN1', 'MCM2', 'MCM4', 'RRM1', 'UNG', 'GINS2', 'MCM6', 'CDCA7', 'DTL', 'PRIM1', 'UHRF1', 'MLF1IP', 'HELLS', 'RFC2', 'RPA2', 'NASP', 'RAD51AP1', 'GMNN', 'WDR76', 'SLBP', 'CCNE2', 'UBR7', 'POLD3', 'MSH2', 'ATAD2', 'RAD51', 'RRM2', 'CDC45', 'CDC6', 'EXO1', 'TIPIN', 'DSCC1', 'BLM', 'CASP8AP2', 'USP1', 'CLSPN', 'POLA1', 'CHAF1B', 'BRIP1', 'E2F8']

g2m_genes = ['HMGB2', 'CDK1', 'NUSAP1', 'UBE2C', 'BIRC5', 'TPX2', 'TOP2A', 'NDC80', 'CKS2', 'NUF2', 'CKS1B', 'MKI67', 'TMPO', 'CENPF', 'TACC3', 'FAM64A', 'SMC4', 'CCNB2', 'CKAP2L', 'CKAP2', 'AURKB', 'BUB1', 'KIF11', 'ANP32E', 'TUBB4B', 'GTSE1', 'KIF20B', 'HJURP', 'CDCA3', 'HN1', 'CDC20', 'TTK', 'CDC25C', 'KIF2C', 'RANGAP1', 'NCAPD2', 'DLGAP5', 'CDCA2', 'CDCA8', 'ECT2', 'KIF23', 'HMMR', 'AURKA', 'PSRC1', 'ANLN', 'LBR', 'CKAP5', 'CENPE', 'CTCF', 'NEK2', 'G2E3', 'GAS2L3', 'CBX5', 'CENPA']
print( len(s_genes), len(g2m_genes) )


s_genes = list( set(adata.var_names) & set(s_genes) )
g2m_genes = list( set(adata.var_names) & set(g2m_genes) )
print( len(s_genes), len(g2m_genes) )


## Create scores and phase marker by scanpy

In [ ]:
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)
adata

In [ ]:
adata.obs.phase[:3]

In [ ]:
!pip install ncvis
import ncvis
vis = ncvis.NCVis()

!pip install trimap 
import trimap
trimap1 = trimap.TRIMAP()

# Visualization is done on subset of cell cycle related genes

In [ ]:
vec_color_by = adata.obs.phase.values

url = 'https://raw.githubusercontent.com/chervov/genes/main/cell_cycle_genes_reactome_extended.txt'
df_list_genes_cc = pd.read_csv(url, header = None)
df_list_genes_cc
mask = genes[1].isin( list( df_list_genes_cc[0].values) )
IX_cc = np.where(mask > 0)[0]
IX_cc, 
len(IX_cc)


from sklearn.decomposition import TruncatedSVD
import time

t00 = time.time()

m3 = m2.copy()#  != 0 # 
m3 = m3[:,IX_cc]

if 0: # drop top expressed genes
    count_drop_top_expressed_genes = 10 # ]: # range(10,100,10):
    v = np.asarray(m2.sum(axis=0)).ravel()
    IX = np.argsort(v )
    if count_made_zeros != 0:
        m3 = m3[:,IX[:-count_drop_top_expressed_genes]] # droping columns - same as make them zeros , but does not create memory alloc prob

t0 = time.time()
svd = TruncatedSVD( n_components=50, n_iter=7, random_state=42)
X = svd.fit_transform(m3)
print(time.time() - t0, 'seconds passed on svd')
t0 = time.time()

import umap
from sklearn import manifold
from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import NMF
from sklearn.decomposition import FastICA
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.random_projection import SparseRandomProjection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD


from collections import OrderedDict
from functools import partial
from matplotlib.ticker import NullFormatter



n_neighbors = 10
n_components = 2
# Set-up manifold methods
LLE = partial(manifold.LocallyLinearEmbedding,
              n_neighbors, n_components, eigen_solver='auto')

methods = OrderedDict()
methods['PCA'] = PCA()
methods['umap'] = umap.UMAP(n_components = n_components)
methods['t-SNE'] = manifold.TSNE(n_components=n_components, init='pca', random_state=0)
methods['NCIvis'] = ncvis.NCVis()
methods['trimap'] = trimap.TRIMAP()
methods['ICA'] = FastICA(n_components=n_components,         random_state=0)
methods['FA'] = FactorAnalysis(n_components=n_components, random_state=0)
methods['LLE'] = LLE(method='standard')
methods['Modified LLE'] = LLE(method='modified')
methods['Isomap'] = manifold.Isomap(n_neighbors, n_components)
methods['MDS'] = manifold.MDS(n_components, max_iter=100, n_init=1)
methods['SE'] = manifold.SpectralEmbedding(n_components=n_components,
                                           n_neighbors=n_neighbors)
methods['NMF'] = NMF(n_components=n_components,  init='random', random_state=0) 
methods['RandProj'] = SparseRandomProjection(n_components=n_components, random_state=42)

rand_trees_embed = make_pipeline(RandomTreesEmbedding(n_estimators=200, random_state=0, max_depth=5), TruncatedSVD(n_components=n_components) )
methods['RandTrees'] = rand_trees_embed
methods['LatDirAll'] = LatentDirichletAllocation(n_components=n_components,  random_state=0)
methods['LTSA'] = LLE(method='ltsa') 
methods['Hessian LLE'] = LLE(method='hessian') 

list_fast_methods = ['PCA','umap','FA', 'ICA','NMF','RandProj','RandTrees']
list_slow_methods = ['t-SNE','LLE','Modified LLE','Isomap','MDS','SE','LatDirAll','LTSA','Hessian LLE']

# transformer = NeighborhoodComponentsAnalysis(init='random',  n_components=2, random_state=0) # Cannot be applied since supervised - requires y 
# methods['LinDisA'] = LinearDiscriminantAnalysis(n_components=n_components)# Cannot be applied since supervised - requires y 


# Create figure
fig = plt.figure(figsize=(25, 16))

# Plot results
c = 0
for i, (label, method) in enumerate(methods.items()):
    #if label in ['umap']: continue
    if 0:
        if label not in  list_fast_methods :
            continue
        
    t0 = time.time()
    try:
        r = method.fit_transform(X.copy())
    except:
        print('Got Exception', label )
        continue 
    t1 = time.time()
    print("%s: %.2g sec" % (label, t1 - t0))
    
    if 1:
        c+=1
        fig.add_subplot(2, 3 , c) 
        sns.scatterplot(x=r[:,0],y=r[:,1]  , hue= vec_color_by )#  cell_lines_tissues_top_truncated, style =cell_lines_tissues_top_truncated , s = 250 )# df['drug'] ,  style=df['dose'])
        plt.title(label )
        plt.legend()
        if c == 6:
            plt.show()
            fig = plt.figure(figsize=(25, 16))
            c = 0


In [ ]:
print(time.time()-t0_start,'secondas passed total')

# Visualization based on marker genes 

In [ ]:
s_genes = ['MCM5', 'PCNA', 'TYMS', 'FEN1', 'MCM2', 'MCM4', 'RRM1', 'UNG', 'GINS2', 'MCM6', 'CDCA7', 'DTL', 'PRIM1', 'UHRF1', 'MLF1IP', 'HELLS', 'RFC2', 'RPA2', 'NASP', 'RAD51AP1', 'GMNN', 'WDR76', 'SLBP', 'CCNE2', 'UBR7', 'POLD3', 'MSH2', 'ATAD2', 'RAD51', 'RRM2', 'CDC45', 'CDC6', 'EXO1', 'TIPIN', 'DSCC1', 'BLM', 'CASP8AP2', 'USP1', 'CLSPN', 'POLA1', 'CHAF1B', 'BRIP1', 'E2F8']

g2m_genes = ['HMGB2', 'CDK1', 'NUSAP1', 'UBE2C', 'BIRC5', 'TPX2', 'TOP2A', 'NDC80', 'CKS2', 'NUF2', 'CKS1B', 'MKI67', 'TMPO', 'CENPF', 'TACC3', 'FAM64A', 'SMC4', 'CCNB2', 'CKAP2L', 'CKAP2', 'AURKB', 'BUB1', 'KIF11', 'ANP32E', 'TUBB4B', 'GTSE1', 'KIF20B', 'HJURP', 'CDCA3', 'HN1', 'CDC20', 'TTK', 'CDC25C', 'KIF2C', 'RANGAP1', 'NCAPD2', 'DLGAP5', 'CDCA2', 'CDCA8', 'ECT2', 'KIF23', 'HMMR', 'AURKA', 'PSRC1', 'ANLN', 'LBR', 'CKAP5', 'CENPE', 'CTCF', 'NEK2', 'G2E3', 'GAS2L3', 'CBX5', 'CENPA']
print( len(s_genes), len(g2m_genes) )
s_genes = list( set(adata.var_names) & set(s_genes) )
g2m_genes = list( set(adata.var_names) & set(g2m_genes) )
print( len(s_genes), len(g2m_genes) )

genes_selected_Tirosh = s_genes + g2m_genes
print(type(genes_selected_Tirosh), len(genes_selected_Tirosh) )


In [ ]:
genes_selected = genes_selected_Tirosh

vec_color_by = adata.obs.phase.values


mask = genes[1].isin( list( genes_selected ) )
IX_cc = np.where(mask > 0)[0]
IX_cc, 

print('Number of selected genes: ', len(IX_cc) )

from sklearn.decomposition import TruncatedSVD
import time

t00 = time.time()

m3 = m2.copy()#  != 0 # 
m3 = m3[:,IX_cc]

if 0: # drop top expressed genes
    count_drop_top_expressed_genes = 10 # ]: # range(10,100,10):
    v = np.asarray(m2.sum(axis=0)).ravel()
    IX = np.argsort(v )
    if count_made_zeros != 0:
        m3 = m3[:,IX[:-count_drop_top_expressed_genes]] # droping columns - same as make them zeros , but does not create memory alloc prob

t0 = time.time()
svd = TruncatedSVD( n_components=50, n_iter=7, random_state=42)
X = svd.fit_transform(m3)
print(time.time() - t0, 'seconds passed on svd')
t0 = time.time()

import umap
from sklearn import manifold
from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import NMF
from sklearn.decomposition import FastICA
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.random_projection import SparseRandomProjection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD


from collections import OrderedDict
from functools import partial
from matplotlib.ticker import NullFormatter



n_neighbors = 10
n_components = 2
# Set-up manifold methods
LLE = partial(manifold.LocallyLinearEmbedding,
              n_neighbors, n_components, eigen_solver='auto')

methods = OrderedDict()
methods['PCA'] = PCA()
methods['umap'] = umap.UMAP(n_components = n_components)
methods['t-SNE'] = manifold.TSNE(n_components=n_components, init='pca', random_state=0)
methods['NCIvis'] = ncvis.NCVis()
methods['trimap'] = trimap.TRIMAP()
methods['ICA'] = FastICA(n_components=n_components,         random_state=0)
methods['FA'] = FactorAnalysis(n_components=n_components, random_state=0)
methods['LLE'] = LLE(method='standard')
methods['Modified LLE'] = LLE(method='modified')
methods['Isomap'] = manifold.Isomap(n_neighbors, n_components)
methods['MDS'] = manifold.MDS(n_components, max_iter=100, n_init=1)
methods['SE'] = manifold.SpectralEmbedding(n_components=n_components,
                                           n_neighbors=n_neighbors)
methods['NMF'] = NMF(n_components=n_components,  init='random', random_state=0) 
methods['RandProj'] = SparseRandomProjection(n_components=n_components, random_state=42)

rand_trees_embed = make_pipeline(RandomTreesEmbedding(n_estimators=200, random_state=0, max_depth=5), TruncatedSVD(n_components=n_components) )
methods['RandTrees'] = rand_trees_embed
methods['LatDirAll'] = LatentDirichletAllocation(n_components=n_components,  random_state=0)
methods['LTSA'] = LLE(method='ltsa') 
methods['Hessian LLE'] = LLE(method='hessian') 

list_fast_methods = ['PCA','umap','FA', 'ICA','NMF','RandProj','RandTrees']
list_slow_methods = ['t-SNE','LLE','Modified LLE','Isomap','MDS','SE','LatDirAll','LTSA','Hessian LLE']

# transformer = NeighborhoodComponentsAnalysis(init='random',  n_components=2, random_state=0) # Cannot be applied since supervised - requires y 
# methods['LinDisA'] = LinearDiscriminantAnalysis(n_components=n_components)# Cannot be applied since supervised - requires y 


# Create figure
fig = plt.figure(figsize=(25, 16))

# Plot results
c = 0
for i, (label, method) in enumerate(methods.items()):
    #if label in ['umap']: continue
    if 0:
        if label not in  list_fast_methods :
            continue
        
    t0 = time.time()
    try:
        r = method.fit_transform(X.copy())
    except:
        print('Got Exception', label )
        continue 
    t1 = time.time()
    print("%s: %.2g sec" % (label, t1 - t0))
    
    if 1:
        c+=1
        fig.add_subplot(2, 3 , c) 
        sns.scatterplot(x=r[:,0],y=r[:,1]  , hue= vec_color_by )#  cell_lines_tissues_top_truncated, style =cell_lines_tissues_top_truncated , s = 250 )# df['drug'] ,  style=df['dose'])
        plt.title(label )
        plt.legend()
        if c == 6:
            plt.show()
            fig = plt.figure(figsize=(25, 16))
            c = 0


# Genes selected by AZ based on ICA (using specifics of the current dataset)

In [ ]:
s_genes = ['BRCA1', 'MRE11', 'DEPDC1B', 'POLQ', 'NDC1', 'MCM10', 'ASPM', 'WDR62', 'MCM6', 'UNG', 'POLD3', 'RBL1', 'ORC1', 'L2HGDH', 'CLSPN', 'MCM5', 'POLE2', 'TRIM9', 'CEP128', 'MYBL2', 'TTI1', 'E2F1', 'CEP152', 'LIG1', 'NEIL3', 'GALNT18', 'FBXO5', 'MCM3', 'HMGCS1', 'CEBPZ', 'SRSF7', 'MASTL', 'CIT', 'STIL', 'HJURP', 'HIST1H1D', 'PKMYT1', 'MIS18BP1', 'GINS2', 'NUP210', 'PCNA', 'SLF1', 'DOCK10', 'USP37', 'BRIP1', 'KIAA0319', 'DNA2', 'ZGRF1', 'SLC7A1', 'DIAPH3', 'TICRR', 'COPS3', 'SSH2', 'ITGB3BP', 'DTL', 'OSBPL10', 'MMS22L', 'CDCA5', 'NPAT', 'NCAPD3', 'CENPU', 'CENPJ', 'SPC25', 'JAZF1', 'ATAD2', 'PAXIP1', 'GALNT14', 'FANCC', 'KIAA1524', 'ZNF704', 'ZNF367', 'E2F7', 'MCM7', 'PLEKHA7', 'CDT1', 'C19orf48', 'TK1', 'CDCA4', 'RRM2', 'MACROD2', 'IQCB1', 'EXO1', 'POLE', 'PLD5', 'GPC6', 'SLC35F3', 'LIN9', 'WWOX', 'KIF18B', 'FANCA', 'ARHGAP11B', 'CENPP', 'FHIT', 'XRCC2', 'RAD54B', 'NUP62CL', 'ZNF43', 'EMP2', 'AC073529.1', 'LINC01036', 'ZFPM2-AS1', 'AC134511.1', 'LINC01572', 'AC016205.1']

g2m_genes = ['PAF1', 'ANLN', 'TACC3', 'CPS1', 'DEPDC1', 'DEPDC1B', 'POLQ', 'SLC12A2', 'ASPM', 'PRR11', 'HMMR', 'GTSE1', 'WDR62', 'SPAG5', 'AURKA', 'TPX2', 'BIRC5', 'KIF4A', 'RANGAP1', 'CEP128', 'ZMYND8', 'TUBD1', 'NEIL3', 'HSPA8', 'FOXM1', 'CEP70', 'SFPQ', 'CDC20', 'NEK2', 'CENPF', 'KIF14', 'CEP89', 'KIF18A', 'CIT', 'TUBA1B', 'STIL', 'HJURP', 'CKS2', 'DLGAP5', 'MIS18BP1', 'SGO1', 'TOP2A', 'CCNB1', 'CENPE', 'DIAPH3', 'TICRR', 'CCNA2', 'MKI67', 'SOGA1', 'SCLT1', 'SPC25', 'POC5', 'CCNB2', 'CDC25C', 'RACGAP1', 'CCNF', 'KIAA1524', 'H2AFZ', 'PLK1', 'TUBA1C', 'TK1', 'CKAP2L', 'BUB1', 'CDK1', 'ARL6IP1', 'SHCBP1', 'RRM2', 'LCORL', 'AURKB', 'PAPPA', 'PRKX', 'CDCA2', 'H1FX', 'WWOX', 'KIF18B', 'PRELID2', 'ARHGAP11B', 'TUBB4B', 'H2AFX', 'TUBB', 'ARHGAP11A', 'PRC1', 'HSPA1B', 'HSPA1A', 'MT-RNR1', 'MXD3', 'AC073529.1', 'MALAT1', 'LINC01572', 'AC016205.1']

print( len(s_genes), len(g2m_genes) )
s_genes = list( set(adata.var_names) & set(s_genes) )
g2m_genes = list( set(adata.var_names) & set(g2m_genes) )
print( len(s_genes), len(g2m_genes) )

genes_selected_AZ = s_genes + g2m_genes
print(type(genes_selected_AZ), len(genes_selected_AZ) )


In [ ]:
genes_selected = genes_selected_AZ

vec_color_by = adata.obs.phase.values


mask = genes[1].isin( list( genes_selected ) )
IX_cc = np.where(mask > 0)[0]
IX_cc, 

print('Number of selected genes: ', len(IX_cc) )

from sklearn.decomposition import TruncatedSVD
import time

t00 = time.time()

m3 = m2.copy()#  != 0 # 
m3 = m3[:,IX_cc]

if 0: # drop top expressed genes
    count_drop_top_expressed_genes = 10 # ]: # range(10,100,10):
    v = np.asarray(m2.sum(axis=0)).ravel()
    IX = np.argsort(v )
    if count_made_zeros != 0:
        m3 = m3[:,IX[:-count_drop_top_expressed_genes]] # droping columns - same as make them zeros , but does not create memory alloc prob

t0 = time.time()
svd = TruncatedSVD( n_components=50, n_iter=7, random_state=42)
X = svd.fit_transform(m3)
print(time.time() - t0, 'seconds passed on svd')
t0 = time.time()

import umap
from sklearn import manifold
from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import NMF
from sklearn.decomposition import FastICA
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.random_projection import SparseRandomProjection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD


from collections import OrderedDict
from functools import partial
from matplotlib.ticker import NullFormatter



n_neighbors = 10
n_components = 2
# Set-up manifold methods
LLE = partial(manifold.LocallyLinearEmbedding,
              n_neighbors, n_components, eigen_solver='auto')

methods = OrderedDict()
methods['PCA'] = PCA()
methods['umap'] = umap.UMAP(n_components = n_components)
methods['t-SNE'] = manifold.TSNE(n_components=n_components, init='pca', random_state=0)
methods['NCIvis'] = ncvis.NCVis()
methods['trimap'] = trimap.TRIMAP()
methods['ICA'] = FastICA(n_components=n_components,         random_state=0)
methods['FA'] = FactorAnalysis(n_components=n_components, random_state=0)
methods['LLE'] = LLE(method='standard')
methods['Modified LLE'] = LLE(method='modified')
methods['Isomap'] = manifold.Isomap(n_neighbors, n_components)
methods['MDS'] = manifold.MDS(n_components, max_iter=100, n_init=1)
methods['SE'] = manifold.SpectralEmbedding(n_components=n_components,
                                           n_neighbors=n_neighbors)
methods['NMF'] = NMF(n_components=n_components,  init='random', random_state=0) 
methods['RandProj'] = SparseRandomProjection(n_components=n_components, random_state=42)

rand_trees_embed = make_pipeline(RandomTreesEmbedding(n_estimators=200, random_state=0, max_depth=5), TruncatedSVD(n_components=n_components) )
methods['RandTrees'] = rand_trees_embed
methods['LatDirAll'] = LatentDirichletAllocation(n_components=n_components,  random_state=0)
methods['LTSA'] = LLE(method='ltsa') 
methods['Hessian LLE'] = LLE(method='hessian') 

list_fast_methods = ['PCA','umap','FA', 'ICA','NMF','RandProj','RandTrees']
list_slow_methods = ['t-SNE','LLE','Modified LLE','Isomap','MDS','SE','LatDirAll','LTSA','Hessian LLE']

# transformer = NeighborhoodComponentsAnalysis(init='random',  n_components=2, random_state=0) # Cannot be applied since supervised - requires y 
# methods['LinDisA'] = LinearDiscriminantAnalysis(n_components=n_components)# Cannot be applied since supervised - requires y 


# Create figure
fig = plt.figure(figsize=(25, 16))

# Plot results
c = 0
for i, (label, method) in enumerate(methods.items()):
    #if label in ['umap']: continue
    if 0:
        if label not in  list_fast_methods :
            continue
        
    t0 = time.time()
    try:
        r = method.fit_transform(X.copy())
    except:
        print('Got Exception', label )
        continue 
    t1 = time.time()
    print("%s: %.2g sec" % (label, t1 - t0))
    
    if 1:
        c+=1
        fig.add_subplot(2, 3 , c) 
        sns.scatterplot(x=r[:,0],y=r[:,1]  , hue= vec_color_by )#  cell_lines_tissues_top_truncated, style =cell_lines_tissues_top_truncated , s = 250 )# df['drug'] ,  style=df['dose'])
        plt.title(label )
        plt.legend()
        if c == 6:
            plt.show()
            fig = plt.figure(figsize=(25, 16))
            c = 0


In [ ]:
print(time.time()-t0_start,'secondas passed total')